In [16]:
#install packages
!pip install numpy torch scikit-learn onnx torchvision

In [17]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
%mkdir "2024_U6_Article_Sentis"
%cd /content/drive/MyDrive/2024_U6_Article_Sentis

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
mkdir: cannot create directory ‘2024_U6_Article_Sentis’: File exists
/content/drive/MyDrive/2024_U6_Article_Sentis


In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import onnx

RANDOM_SEED = 7
torch.manual_seed(RANDOM_SEED)

dataset = 'points_xrhands.csv'
model_save_path = 'xrhands_gesture_classification.pth'
NUM_CLASSES = 4

X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (28 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int64', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Convert numpy arrays to PyTorch tensors and create DataLoaders
train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)

class KeypointClassifier(nn.Module):
    def __init__(self, input_size=28*2, num_classes=NUM_CLASSES):
        super(KeypointClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(),
            nn.Dropout(0.2),

            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.model(x)

model = KeypointClassifier()
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

best_accuracy = 0
patience = 50
counter = 0

for epoch in range(500):
    train_loss = train(model, train_loader, criterion, optimizer)
    test_loss, accuracy = evaluate(model, test_loader, criterion)

    print(f'Epoch: {epoch+1}, Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        counter = 0
        torch.save(model.state_dict(), model_save_path)
        print(f'Model saved with accuracy: {best_accuracy:.2f}%')
    else:
        counter += 1

#validation
model.load_state_dict(torch.load(model_save_path))
val_loss, val_acc = evaluate(model, test_loader, criterion)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')

#inference
test_sample = torch.FloatTensor(X_test[0]).unsqueeze(0)
with torch.no_grad():
    logits = model(test_sample)
    probabilities = F.softmax(logits, dim=1)

print("Probabilities:", probabilities.squeeze().numpy())
print("Predicted class:", probabilities.argmax().item())

KeypointClassifier(
  (model): Sequential(
    (0): Linear(in_features=56, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=64, out_features=32, bias=True)
    (13): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): LeakyReLU(negative_slope=0.01)
    (15): Dropout(p=0.2, inplace=False)
    (16): Linear(in_features=32, out

<ipython-input-18-d9da6a20dd85>:111: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


In [19]:
#onnx export
dummy_input = torch.randn(1, 28*2)
onnx_file_path = "xrhands_gesture_classification.onnx"

torch.onnx.export(model,
                  dummy_input,
                  onnx_file_path,
                  export_params=True,
                  opset_version=15,
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'])

onnx_model = onnx.load(onnx_file_path)
onnx.checker.check_model(onnx_model)